In [1]:
# Importing required items
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import pandas as pd
import shutil
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
# Confirm the connection to the GPU with tensorflow
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul 12 00:00:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W /  70W |    387MiB / 15360MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Determine the available RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [5]:
# Defining the training and testing directories
training_data_dir = '/content/drive/MyDrive/images/train'
test_data_dir = '/content/drive/MyDrive/images/test'



In [6]:
# Defining hyperparameters
img_width, img_height = 150, 150
batch_size = 15
epochs = 30
num_classes = 12  # Number of different types in dataset

In [7]:
# Defining data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [8]:
test_datagen = ImageDataGenerator(rescale=1/255)

In [9]:
# Defining training generator
train_generator = train_datagen.flow_from_directory(
    training_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4750 images belonging to 12 classes.


In [10]:
# Defining the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [11]:
# Adjusting learning rate and change optimization algorithm
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

In [12]:
# Compiling the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

In [13]:
# Optimizing and improving memory usage
tf.config.optimizer.set_jit(True)

In [14]:
# Training the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
)

Epoch 1/30
316/316 [==============================] - 1258s 4s/step - loss: 2.1949 - accuracy: 0.2522
Epoch 2/30
316/316 [==============================] - 88s 277ms/step - loss: 1.5637 - accuracy: 0.4629
Epoch 3/30
316/316 [==============================] - 85s 269ms/step - loss: 1.2690 - accuracy: 0.5668
Epoch 4/30
316/316 [==============================] - 85s 268ms/step - loss: 1.1332 - accuracy: 0.6215
Epoch 5/30
316/316 [==============================] - 83s 264ms/step - loss: 1.0198 - accuracy: 0.6558
Epoch 6/30
316/316 [==============================] - 85s 269ms/step - loss: 0.9593 - accuracy: 0.6735
Epoch 7/30
316/316 [==============================] - 83s 264ms/step - loss: 0.9132 - accuracy: 0.6914
Epoch 8/30
316/316 [==============================] - 84s 266ms/step - loss: 0.8332 - accuracy: 0.7193
Epoch 9/30
316/316 [==============================] - 85s 269ms/step - loss: 0.7828 - accuracy: 0.7392
Epoch 10/30
316/316 [==============================] - 84s 265ms/step - lo

In [15]:
# Saving the trained model
model.save('project4_model.h5')

In [16]:
# Loading the trained model
model = tf.keras.models.load_model('project4_model.h5')

In [17]:
# Creating an ImageDataGenerator for the testing dataset
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [18]:
# Loading the filenames of the images in the testing dataset
test_filenames = os.listdir(test_data_dir)

In [19]:
# Creating a dataframe with the filenames
test_df = pd.DataFrame({'filename': test_filenames})
print(test_filenames)
print(len(test_filenames))

['0021e90e4.png', '003d61042.png', '007b3da8b.png', '0086a6340.png', '00c47e980.png', '00d090cde.png', '00ef713a8.png', '01291174f.png', '026716f9b.png', '02cfeb38d.png', '03566743d.png', '03a2ee656.png', '03e322a29.png', '03ef36742.png', '043449b0b.png', '0437393b1.png', '04814f36d.png', '05341a8a6.png', '060450d79.png', '060f1dc84.png', '0625f063b.png', '063363305.png', '06d12f6fa.png', '071cb3ece.png', '0751c0bbc.png', '07e62f903.png', '085974290.png', '0885e7690.png', '089ad62a7.png', '08d591441.png', '0911d3dee.png', '099b961ec.png', '0a64e3e6c.png', '0ad9e7dfb.png', '0ae6668fa.png', '0bf7bfb05.png', '0c27cf05f.png', '0c4199daa.png', '0c45ace27.png', '0c51bf229.png', '0c5f6c493.png', '0caeda5df.png', '0d117d910.png', '0d31e6602.png', '0dba99002.png', '0e8492cb1.png', '0ebf8f2f4.png', '0ee4ad224.png', '0f6cbe5e8.png', '0fb233ad6.png', '115f93ecc.png', '116b136de.png', '1191ba346.png', '11d3f68ff.png', '122913909.png', '12625488b.png', '126a71ce0.png', '1312065a5.png', '1364b297a.pn

In [20]:
# Generating the test data generator from the dataframe
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_data_dir,
    x_col='filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

Found 794 validated image filenames.


In [21]:
# Predicting the classes for the test images
predictions = model.predict(test_generator)

53/53 [==============================] - 6s 100ms/step


In [22]:
# Maping predicted class indices to class labels
class_labels = list(train_generator.class_indices.keys())
predicted_classes = [class_labels[prediction.argmax()] for prediction in predictions]

In [23]:
# Printing the predicted classes
print(predicted_classes)

['Small-flowered Cranesbill', 'Fat Hen', 'Sugar beet', 'Common Chickweed', 'Sugar beet', 'Loose Silky-bent', 'Common Chickweed', 'Fat Hen', 'Loose Silky-bent', 'Loose Silky-bent', 'Maize', 'Small-flowered Cranesbill', 'Sugar beet', 'Scentless Mayweed', 'Sugar beet', 'Fat Hen', 'Scentless Mayweed', 'Scentless Mayweed', 'Common Chickweed', 'Fat Hen', 'Common Chickweed', 'Small-flowered Cranesbill', 'Shepherds Purse', 'Sugar beet', 'Sugar beet', 'Maize', 'Sugar beet', 'Scentless Mayweed', 'Common Chickweed', 'Sugar beet', 'Common Chickweed', 'Shepherds Purse', 'Loose Silky-bent', 'Maize', 'Loose Silky-bent', 'Black-grass', 'Loose Silky-bent', 'Loose Silky-bent', 'Common Chickweed', 'Maize', 'Loose Silky-bent', 'Common wheat', 'Common Chickweed', 'Small-flowered Cranesbill', 'Sugar beet', 'Cleavers', 'Maize', 'Scentless Mayweed', 'Maize', 'Small-flowered Cranesbill', 'Loose Silky-bent', 'Sugar beet', 'Cleavers', 'Loose Silky-bent', 'Maize', 'Fat Hen', 'Charlock', 'Small-flowered Cranesbill

In [24]:
# Adding the predicted classes to the dataframe
test_df['predictions'] = predicted_classes
test_df.head()

,filename,predictions
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Fat Hen
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Sugar beet


In [25]:
# Creating the predicted subfolders for each type
predicted_dir = '/content/drive/MyDrive/images/predictions'
os.makedirs(predicted_dir, exist_ok=True)
for class_label in class_labels:
    class_dir = os.path.join(predicted_dir, class_label)
    os.makedirs(class_dir, exist_ok=True)

In [26]:
# Copying the images to their predicted folders
for filename, predicted_class in zip(test_filenames, predicted_classes):
    source_path = os.path.join(test_data_dir, filename)
    destination_path = os.path.join(predicted_dir, predicted_class, filename)
    shutil.copy(source_path, destination_path)

print("Images have been copyed to their respective predicted folders.")

Images have been copyed to their respective predicted folders.
